In [1]:
import pandas as pd
import os
import plotly

In [2]:
os.getcwd()

'c:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\notebooks'

In [4]:
tbidata=pd.read_csv('C:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\data\\tbi_data_files.csv')
#tbidata

,donor_id,donor_name,specimen_id,specimen_name,rna_well,rna_integrity_number,structure_id,structure_acronym,structure_name,rnaseq_profile_id,rnaseq_total_reads,rnaseq_percent_reads_aligned_to_mrna,rnaseq_percent_reads_aligned_to_ncrna,rnaseq_percent_reads_aligned_to_genome_only,gene_level_fpkm_file_link,anonymized_bam_file_link,anonymized_bam_index_file_link,bigwig_file_link
0,309335438,H14.09.001,309357595,H14.09.001.HIP.05,320630832,7.3,10294,HIP,hippocampus (hippocampal formation),496100314,32275545,31.7,7.11,48.1,/api/v2/well_known_file_download/496128434,/api/v2/well_known_file_download/501023521,/api/v2/well_known_file_download/501023519,/api/v2/well_known_file_download/501056759
1,309335438,H14.09.001,309357596,H14.09.001.PCx.01,320630834,7.2,10557,FWM,white matter of forebrain,496100278,32515376,29.0,8.21,49.5,/api/v2/well_known_file_download/496106975,/api/v2/well_known_file_download/500938472,/api/v2/well_known_file_download/500938470,/api/v2/well_known_file_download/500941271
2,309335438,H14.09.001,309357596,H14.09.001.PCx.01,320630836,7.1,10208,PCx,parietal neocortex,496100290,34426215,29.1,6.59,52.1,/api/v2/well_known_file_download/496555481,/api/v2/well_known_file_download/500941225,/api/v2/well_known_file_download/500941223,/api/v2/well_known_file_download/500941630
3,309335438,H14.09.001,309357599,H14.09.001.TCx.01,320630838,7.3,10235,TCx,temporal neocortex,496100279,31714711,31.4,6.97,48.7,/api/v2/well_known_file_download/496106814,/api/v2/well_known_file_download/500936841,/api/v2/well_known_file_download/500936839,/api/v2/well_known_file_download/500941005
4,309335439,H14.09.002,309357603,H14.09.002.HIP.01,320630842,6.4,10294,HIP,hippocampus (hippocampal formation),496100281,33402591,29.5,7.21,50.6,/api/v2/well_known_file_download/496106950,/api/v2/well_known_file_download/500938896,/api/v2/well_known_file_download/500938894,/api/v2/well_known_file_download/500941300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,467056408,H15.09.109,467179122,H15.09.109.TCx.01,482655796,6.6,10235,TCx,temporal neocortex,496100650,41485057,26.8,6.50,54.9,/api/v2/well_known_file_download/496372690,/api/v2/well_known_file_download/501307834,/api/v2/well_known_file_download/501307832,/api/v2/well_known_file_download/501308010
373,467056409,H15.09.110,467179125,H15.09.110.HIP.01,482655798,6.8,10294,HIP,hippocampus (hippocampal formation),496100664,37271308,30.8,7.17,50.7,/api/v2/well_known_file_download/496551665,/api/v2/well_known_file_download/501305355,/api/v2/well_known_file_download/501305353,/api/v2/well_known_file_download/501307754
374,467056409,H15.09.110,467179128,H15.09.110.PCx.01,482655800,7.0,10557,FWM,white matter of forebrain,496100657,27665286,30.0,8.34,49.6,/api/v2/well_known_file_download/496549729,/api/v2/well_known_file_download/501265455,/api/v2/well_known_file_download/501265453,/api/v2/well_known_file_download/501269273
375,467056409,H15.09.110,467179128,H15.09.110.PCx.01,482655802,7.0,10208,PCx,parietal neocortex,496100653,30395673,31.8,7.63,48.3,/api/v2/well_known_file_download/496358437,/api/v2/well_known_file_download/501269458,/api/v2/well_known_file_download/501269456,/api/v2/well_known_file_download/501271208


In [ ]:
# # read in files needed to be downloaded to get counts, create dataframe from these files, takes 20min

# file_links=tbidata['gene_level_fpkm_file_link'].to_list()
# for link in file_links:
#     if link==file_links[0]:
#         print("http://api.brain-map.org"+link)
#         df_gene_counts=pd.read_csv("http://api.brain-map.org"+link,delimiter='\\t')
#         df_gene_counts['link']=link
#     else:
#         file_link="http://api.brain-map.org"+link
#         new_file=pd.read_csv(file_link,delimiter='\\t')
#         new_file['link']=link
#         df_gene_counts=pd.concat([df_gene_counts,new_file])


# # df_gene_counts


In [6]:
# #save df as csv as to not repeat process
# df_gene_counts.to_csv('C:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\data\\df_gene_counts.csv')

In [ ]:
df_gene_counts=pd.read_csv('C:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\data\\df_gene_counts.csv')


In [ ]:
#df_gene_counts[df_gene_counts['gene_id']==3507]

In [7]:
genes=pd.read_csv('C:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\data\\gene_expression_matrix_2016-03-03\\rows-genes.csv')


In [8]:
merge_table=df_gene_counts.merge(tbidata,left_on='link',right_on='gene_level_fpkm_file_link')

In [9]:
merge_table.columns

Index(['gene_id', 'transcript_id(s)', 'length', 'effective_length',
       'expected_count', 'TPM', 'FPKM', 'link', 'donor_id', 'donor_name',
       'specimen_id', 'specimen_name', 'rna_well', 'rna_integrity_number',
       'structure_id', 'structure_acronym', 'structure_name',
       'rnaseq_profile_id', 'rnaseq_total_reads',
       'rnaseq_percent_reads_aligned_to_mrna',
       'rnaseq_percent_reads_aligned_to_ncrna',
       'rnaseq_percent_reads_aligned_to_genome_only',
       'gene_level_fpkm_file_link', 'anonymized_bam_file_link',
       'anonymized_bam_index_file_link', 'bigwig_file_link'],
      dtype='object')

In [10]:
genes=pd.read_csv('C:\\Users\\dyeca\\OneDrive\\Documents\\capstone\\aging_dementia_tbi\\data\\gene_expression_matrix_2016-03-03\\rows-genes.csv')
genes
mapping_dict_gene=dict(zip(genes['gene_entrez_id'],genes['gene_id']))
#mapping_dict_gene

In [11]:
total_counts=merge_table[['expected_count','TPM','FPKM','donor_id', 'donor_name',
       'specimen_id', 'specimen_name','structure_id', 'structure_acronym', 'structure_name',
       'rnaseq_profile_id','gene_id']]
total_counts['gene_id_mapped']=total_counts['gene_id'].map(mapping_dict_gene)
#total_counts

C:\Users\dyeca\AppData\Local\Temp\ipykernel_2512\789333118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_counts['gene_id_mapped']=total_counts['gene_id'].map(mapping_dict_gene)


In [12]:
total_counts[total_counts['gene_id_mapped'].isna()]

,expected_count,TPM,FPKM,donor_id,donor_name,specimen_id,specimen_name,structure_id,structure_acronym,structure_name,rnaseq_profile_id,gene_id,gene_id_mapped
33446,0.0,0.00,0.00,309335438,H14.09.001,309357595,H14.09.001.HIP.05,10294,HIP,hippocampus (hippocampal formation),496100314,3502,NaN
33451,0.0,0.00,0.00,309335438,H14.09.001,309357595,H14.09.001.HIP.05,10294,HIP,hippocampus (hippocampal formation),496100314,3507,NaN
83729,0.0,0.00,0.00,309335438,H14.09.001,309357596,H14.09.001.PCx.01,10557,FWM,white matter of forebrain,496100278,3502,NaN
83734,0.0,0.00,0.00,309335438,H14.09.001,309357596,H14.09.001.PCx.01,10557,FWM,white matter of forebrain,496100278,3507,NaN
134012,0.0,0.00,0.00,309335438,H14.09.001,309357596,H14.09.001.PCx.01,10208,PCx,parietal neocortex,496100290,3502,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18839293,0.0,0.00,0.00,467056409,H15.09.110,467179128,H15.09.110.PCx.01,10557,FWM,white matter of forebrain,496100657,3507,NaN
18889571,0.0,0.00,0.00,467056409,H15.09.110,467179128,H15.09.110.PCx.01,10208,PCx,parietal neocortex,496100653,3502,NaN
18889576,0.0,0.00,0.00,467056409,H15.09.110,467179128,H15.09.110.PCx.01,10208,PCx,parietal neocortex,496100653,3507,NaN
18939854,0.0,0.00,0.00,467056409,H15.09.110,467179131,H15.09.110.TCx.01,10235,TCx,temporal neocortex,496100646,3502,NaN


In [13]:
unmapped_genes=list(total_counts[total_counts['gene_id_mapped'].isna()]['gene_id'].unique())

In [14]:
total_counts=total_counts.drop(total_counts[total_counts['gene_id'].isin(unmapped_genes)].index)

In [15]:
ct_matrix=total_counts[['gene_id_mapped','rnaseq_profile_id','expected_count']].pivot(index='gene_id_mapped',columns='rnaseq_profile_id', values='expected_count')
#ct_matrix

In [16]:
ct_matrix

rnaseq_profile_id,488395315,496100277,496100278,496100279,496100281,496100283,496100284,496100285,496100287,496100288,...,496100661,496100663,496100664,496100665,496100666,496100667,496100669,496100670,496100671,496100672
gene_id_mapped,,,,,,,,,,,,,,,,,,,,,
499304660.0,12.50,1.32,0.00,0.00,0.00,0.00,0.00,3.47,4.39,0.0,...,10.28,11.13,0.00,12.92,8.31,0.00,0.00,3.60,0.00,0.00
499304661.0,70.66,194.17,89.89,81.22,83.07,104.23,104.34,78.86,74.60,135.9,...,135.14,100.81,108.51,105.28,108.89,145.39,201.28,140.84,105.96,115.03
499304662.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
499304663.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
499304664.0,0.50,0.46,0.00,2.76,0.00,0.37,2.01,1.50,0.00,2.5,...,0.00,1.00,0.36,0.69,2.22,1.87,1.65,0.00,0.80,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499355059.0,337.00,188.00,242.00,364.00,368.00,420.00,329.00,288.00,265.00,511.0,...,432.00,349.00,307.00,639.00,482.00,481.00,343.00,388.00,467.00,329.00
499355060.0,0.00,0.00,1.00,0.00,2.00,2.00,0.00,3.00,0.00,4.0,...,1.00,0.00,0.00,2.00,2.00,0.00,0.00,1.00,5.00,1.00
499355061.0,5658.00,5006.00,4104.99,4788.00,4350.00,5922.00,3818.00,3806.79,5200.00,10299.0,...,6792.00,7016.00,6054.00,14007.00,8931.00,7448.00,6343.00,6841.00,7499.00,6235.00
